# Google Brain - Ventilator Pressure Prediction / 人工呼吸器の圧力予測

**Simulate a ventilator connected to a sedated patient's lung / 鎮静状態の患者の肺に接続された人工呼吸器をシミュレートする**

## Overview / 概要

What do doctors do when a patient has trouble breathing? They use a ventilator to pump oxygen into a sedated patient's lungs via a tube in the windpipe. But mechanical ventilation is a clinician-intensive procedure, a limitation that was prominently on display during the early days of the COVID-19 pandemic. At the same time, developing new methods for controlling mechanical ventilators is prohibitively expensive, even before reaching clinical trials. High-quality simulators could reduce this barrier.

患者が呼吸困難に陥ったとき、医師は何をするでしょうか？人工呼吸器を使って、鎮静状態の患者の肺に、気管内のチューブを介して酸素を送り込みます。しかし、人工呼吸器は臨床医の手が必要であり、その限界はCOVID-19パンデミックの初期に顕著に現れました。また、人工呼吸器を制御する新しい方法を開発するには、臨床試験に至る前段階であっても莫大な費用がかかります。高品質のシミュレータはこの障壁を減らすことができます。

Current simulators are trained as an ensemble, where each model simulates a single lung setting. However, lungs and their attributes form a continuous space, so a parametric approach must be explored that would consider the differences in patient lungs.

現在のシミュレータは、各モデルが単一の肺の設定をシミュレートするアンサンブルモデルとしてトレーニングされています。しかし、肺とその周辺組織は連続した空間を形成しているため、患者の肺の違いを考慮したパラメトリックなアプローチを検討する必要があります。

Partnering with Princeton University, the team at Google Brain aims to grow the community around machine learning for mechanical ventilation control. They believe that neural networks and deep learning can better generalize across lungs with varying characteristics than the current industry standard of PID controllers.

Google Brainのチームはプリンストン大学と提携し、人工呼吸制御のための機械学習に関するコミュニティの発展を目指しています。ニューラルネットワークと深層学習は、現在の業界標準であるPID制御よりも、さまざまな特性を持つ肺をよりよく一般化できると考えています。

In this competition, you’ll simulate a ventilator connected to a sedated patient's lung. The best submissions will take lung attributes compliance and resistance into account.

このコンペティションでは、鎮静状態の患者の肺に接続された人工呼吸器のシミュレーションを行います。最良の提出物は、肺の特性である追従性や抵抗を考慮したものになります。

If successful, you'll help overcome the cost barrier of developing new methods for controlling mechanical ventilators. This will pave the way for algorithms that adapt to patients and reduce the burden on clinicians during these novel times and beyond. As a result, ventilator treatments may become more widely available to help patients breathe.

成功すれば、人工呼吸器を制御する新しい方法を開発する際のコスト面での障壁を克服することができます。これにより、患者に適応したアルゴリズムへの道が開かれ、この斬新な時代とそれ以降の時代において、臨床医の負担を軽減することができます。その結果、患者さんの呼吸を助けるための人工呼吸器の治療法がより広く普及することになるかもしれません。

## Libraries / ライブラリ

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

## Data / データ

The ventilator data used in this competition was produced using a modified open-source ventilator connected to an artificial bellows test lung via a respiratory circuit. The diagram below illustrates the setup, with the two control inputs highlighted in green and the state variable (airway pressure) to predict in blue. The first control input is a continuous variable from 0 to 100 representing the percentage the inspiratory solenoid valve is open to let air into the lung (i.e., 0 is completely closed and no air is let in and 100 is completely open). The second control input is a binary variable representing whether the exploratory valve is open (1) or closed (0) to let air out.

このコンペティションで使用した人工呼吸器のデータは、オープンソースの人工呼吸器を改造し、呼吸回路を介して人工的な試験肺に接続して作成しました。下図はその構成を示したもので、2つの制御入力を緑で、予測する状態変数（気道圧）を青で示しています。1つ目の制御入力は0〜100の連続変数で、空気を肺に入れるために吸気電磁弁を開く割合を表します（すなわち、0は完全に閉じて空気を入れず、100は完全に開きます）。2つ目の制御入力は、空気を出すための排気電磁弁が開いている（1）か閉じている（0）かを表す二値変数です。

In this competition, participants are given numerous time series of breaths and will learn to predict the airway pressure in the respiratory circuit during the breath, given the time series of control inputs.

このコンペティションでは、参加者に多数の呼吸の時系列データが与えられ、制御入力の時系列が与えられた場合に、呼吸時の呼吸回路内の気道圧を予測することを学習します。

![](https://raw.githubusercontent.com/google/deluca-lung/main/assets/2020-10-02%20Ventilator%20diagram.svg)

Each time series represents an approximately 3-second breath. The files are organized such that each row is a time step in a breath and gives the two control signals, the resulting airway pressure, and relevant attributes of the lung, described below.

各時系列データは約3秒の呼吸を表しています。ファイルは各行が呼吸の時間ステップとなるように構成されており、2つの制御信号、その結果としての気道圧、および以下に述べる肺の関連属性が与えられています。

**Files / ファイル**
- train.csv - the training set / トレーニングセット
- test.csv - the test set / テストセット
- sample_submission.csv - a sample submission file in the correct format / 正しい形式のサンプル提出ファイル

**Columns / 列**
- id - globally-unique time step identifier across an entire file / ファイル全体で一意のタイムステップ識別子
- breath_id - globally-unique time step for breaths / 呼吸の一意な識別子
- R - lung attribute indicating how restricted the airway is (in cmH2O/L/S). Physically, this is the change in pressure per change in flow (air volume per time). Intuitively, one can imagine blowing up a balloon through a straw. We can change R by changing the diameter of the straw, with higher R being harder to blow. / 気道がどの程度制限されているかを示す肺属性（単位：cmH2O/L/S）。物理的には、流量（時間当たりの空気量）の変化に対する圧力の変化です。直感的には、ストローで風船を膨らませるようなイメージです。ストローの直径を変えることでRを変化させることができ、Rが大きいほど吹きにくくなります。
- C - lung attribute indicating how compliant the lung is (in mL/cmH2O). Physically, this is the change in volume per change in pressure. Intuitively, one can imagine the same balloon example. We can change C by changing the thickness of the balloon’s latex, with higher C having thinner latex and easier to blow. / 肺の適合性を示す肺属性（単位：mL/cmH2O）。物理的には、圧力の変化に対する体積の変化を表します。直感的には、同じ風船の例を想像してください。風船のラテックスの厚さを変えることでCを変化させることができます。Cが大きいほどラテックスが薄く、吹きやすくなります。
- time_step - the actual time stamp. / 実際のタイムスタンプ
- u_in - the control input for the inspiratory solenoid valve. Ranges from 0 to 100. / 吸気ソレノイドバルブの制御入力です。0～100の範囲で設定できます。
- u_out - the control input for the exploratory solenoid valve. Either 0 or 1. / 排気ソレノイドバルブの制御入力です。0または1のいずれかです。
- pressure - the airway pressure measured in the respiratory circuit, measured in cmH2O. / 呼吸回路で測定された気道の圧力で、単位はcmH2Oです。

In [ ]:
path = '/kaggle/input/ventilator-pressure-prediction/'
os.listdir(path)

In [ ]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
sample = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
# train.csv
train

In [ ]:
# test.csv
test

In [ ]:
# sample_submission.csv
sample

## Exploratory Data Analysis / 探索的データ解析

### Whole data / データ全体

In [ ]:
alldata = pd.concat([train, test]).reset_index(drop=True)

alldata['data'] = np.nan
alldata.iloc[:len(train), -1] = 'train'
alldata.iloc[len(train):, -1] = 'test'

alldata.info()

In [ ]:
alldata[alldata.columns[:]].describe().round(2)

**pressure**

In [ ]:
# histgram of pressure / 気動圧のヒストグラム
plt.figure(figsize = (10,5))
train['pressure'].hist(bins=50)
plt.show()

**R**

Lung attribute indicating how restricted the airway is (in cmH2O/L/S). Physically, this is the change in pressure per change in flow (air volume per time). Intuitively, one can imagine blowing up a balloon through a straw. We can change R by changing the diameter of the straw, with higher R being harder to blow. / 気道がどの程度制限されているかを示す肺属性（単位：cmH2O/L/S）。物理的には、流量（時間当たりの空気量）の変化に対する圧力の変化です。直感的には、ストローで風船を膨らませるようなイメージです。ストローの直径を変えることでRを変化させることができ、Rが大きいほど吹きにくくなります。

In [ ]:
sns.countplot(data=alldata, x='R', hue='data')
plt.show()

**C**

Lung attribute indicating how compliant the lung is (in mL/cmH2O). Physically, this is the change in volume per change in pressure. Intuitively, one can imagine the same balloon example. We can change C by changing the thickness of the balloon’s latex, with higher C having thinner latex and easier to blow. / 肺の適合性を示す肺属性（単位：mL/cmH2O）。物理的には、圧力の変化に対する体積の変化を表します。直感的には、同じ風船の例を想像してください。風船のラテックスの厚さを変えることでCを変化させることができます。Cが大きいほどラテックスが薄く、吹きやすくなります。

In [ ]:
sns.countplot(data=alldata, x='C', hue='data')
plt.show()

**time_step** 

The actual time stamp. / 実際のタイムスタンプ

In [ ]:
plt.figure(figsize = (10,5))
sns.histplot(data=alldata, x='time_step', hue='data', bins=20)
plt.show()

**u_in**

The control input for the inspiratory solenoid valve. Ranges from 0 to 100. / 吸気ソレノイドバルブの制御入力です。0～100の範囲で設定できます。

In [ ]:
plt.figure(figsize = (10,5))
sns.histplot(data=alldata, x='u_in', hue='data', bins=20)
plt.show()

**u_out**

The control input for the exploratory solenoid valve. Either 0 or 1. / 排気ソレノイドバルブの制御入力です。0または1のいずれかです。

In [ ]:
sns.countplot(data=alldata, x='u_out', hue='data')
plt.show()

### Sample breaths / 呼吸データの例

In [ ]:
# breath_id: 1-10
for i in range(10):
    breath = train.loc[train['breath_id']==i+1].reset_index(drop=True)
    print('breath_id:', i+1)
    breath[['u_in', 'u_out', 'pressure']].plot()
    plt.show()

## Submission / サブミッション

In [ ]:
train_X = train[train.columns[2:-1]]
train_y = train['pressure']
test_X = test[test.columns[2:]]

train_X = pd.get_dummies(train_X, columns=['R', 'C', 'u_out'])
test_X = pd.get_dummies(test_X, columns=['R', 'C', 'u_out'])

In [ ]:
from sklearn.linear_model import LinearRegression

reg  =  LinearRegression()
reg.fit(train_X, train_y)
sample['pressure'] = reg.predict(test_X)
sample.to_csv('submission.csv',index=False)